#Introduction

This notebook has all the code you need to create your own chatbot with custom knowledge base using GPT-3.

Follow the instructions for each steps and then run the code sample. In order to run the code, you need to press "play" button near each code sample.

#Download the data for your custom knowledge base
For the demonstration purposes we are going to use ----- as our knowledge base. You can download them to your local folder from the github repository by running the code below.
Alternatively, you can put your own custom data into the local folder.

In [1]:
! git clone https://github.com/irina1nik/context_data.git

Cloning into 'context_data'...


remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 30 (delta 4), reused 4 (delta 4), pack-reused 23
Receiving objects: 100% (30/30), 12.58 KiB | 348.00 KiB/s, done.
Resolving deltas: 100% (13/13), done.


# Install the dependicies
Run the code below to install the depencies we need for our functions

In [2]:
%pip install llama-index==0.5.6
%pip install langchain==0.0.148

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.0/165.0 KB 2.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.3/794.3 KB 13.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 KB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 KB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 31.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 27.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 KB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.9/773.9 KB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Define the functions
The following code defines the functions we need to construct the index and query it

In [10]:
from llama_index import SimpleDirectoryReader, GPTListIndex, readers, GPTSimpleVectorIndex, LLMPredictor, PromptHelper, ServiceContext
from langchain import OpenAI
import sys
import os
from IPython.display import Markdown, display

def construct_index(directory_path):
    # set maximum input size
    max_input_size = 4096
    # set number of output tokens
    num_outputs = 2000
    # set maximum chunk overlap
    max_chunk_overlap = 20
    # set chunk size limit
    chunk_size_limit = 600

    # define prompt helper
    prompt_helper = PromptHelper(max_input_size, num_outputs, max_chunk_overlap, chunk_size_limit=chunk_size_limit)

    # define LLM
    llm_predictor = LLMPredictor(llm=OpenAI(temperature=0.2, model_name="gpt-4-1106-preview", max_tokens=num_outputs))

    documents = SimpleDirectoryReader(directory_path).load_data()

    service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)
    index = GPTSimpleVectorIndex.from_documents(documents, service_context=service_context)

    index.save_to_disk('index.json')

    return index

def ask_ai():
    index = GPTSimpleVectorIndex.load_from_disk('index.json')
    while True:
        query = input("What do you want to ask? ")
        response = index.query(query)
        display(Markdown(f"Response: <b>{response.response}</b>"))

# Set OpenAI API Key
You need an OPENAI API key to be able to run this code.

If you don't have one yet, get it by [signing up](https://platform.openai.com/overview). Then click your account icon on the top right of the screen and select "View API Keys". Create an API key.

Then run the code below and paste your API key into the text input.

In [11]:
os.environ["OPENAI_API_KEY"] = input("Paste your OpenAI key here and hit enter:")

#Construct an index
Now we are ready to construct the index. This will take every file in the folder 'data', split it into chunks, and embed it with OpenAI's embeddings API.

**Notice:** running this code will cost you credits on your OpenAPI account ($0.02 for every 1,000 tokens). If you've just set up your account, the free credits that you have should be more than enough for this experiment.

In [12]:
construct_index("context_data/data")

/home/tristan/.local/lib/python3.10/site-packages/langchain/llms/openai.py:165: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/home/tristan/.local/lib/python3.10/site-packages/langchain/llms/openai.py:672: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 910815 tokens


#Ask questions
It's time to have fun and test our AI. Run the function that queries GPT and type your question into the input.

If you've used the provided example data for your custom knowledge base, here are a few questions that you can ask:
1. Why people cook at home? Make classification
2. Make classification about what frustrates people about cooking?
3. Brainstorm marketing campaign ideas for an air fryer that would appeal people that cook at home
4. Which kitchen appliences people use most often?
5. What people like about cooking at home?

In [13]:
ask_ai()

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 4314 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 9 tokens


Response: <b>

The Bellman-Ford algorithm is an algorithm for finding the shortest paths from a single source vertex to all other vertices in a weighted, directed graph. It works by iteratively relaxing the edges of the graph, and can detect negative-weight cycles. It runs in O(VE) time, where V is the number of vertices and E is the number of edges. It can be used to find the critical path in a PERT chart by either negating the edge weights and running DAG-SHORTEST-PATHS, or running DAG-SHORTEST-PATHS but replacing <1= by <1= in line 2 of INITIALIZE-SINGLE-SOURCE and <>= by <<= in the RELAX procedure.</b>

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 4572 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 12 tokens


Response: <b>

Divide and conquer algorithms are a type of algorithm that divide a problem into smaller subproblems, solve the subproblems, and then combine the solutions to the subproblems to solve the original problem. This technique dates back to at least 1202, when it was used by Leonardo Bonacci (also known as Fibonacci) to study Fibonacci numbers. It has since been used in many applications, such as matrix multiplication, chip testing, and Monge arrays.

The divide and conquer technique can be used to solve recurrences, such as the Fibonacci recurrence, by using the method of generating functions. It can also be used to identify good chips in a set of supposedly identical integrated-circuit chips, by using a recursive algorithm that reduces the problem to one of nearly half the size. The master method and the Akra-Bazzi method are two general methods for solving recurrences arising from the analysis of divide-and-conquer algorithms.

In general, divide and conquer algorithms can be used to solve problems more efficiently than other methods, as they break down the problem into smaller subproblems that can be solved more quickly.</b>

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 4602 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 12 tokens


Response: <b>

// Divide and Conquer Algorithm

// Input: Array A of size n

// Output: Result of the algorithm

function divideAndConquer(A, n) {
  // Base case
  if (n == 1) {
    return A[0];
  }
  
  // Divide
  mid = n/2;
  left = A[0..mid-1];
  right = A[mid..n-1];
  
  // Conquer
  leftResult = divideAndConquer(left, mid);
  rightResult = divideAndConquer(right, n-mid);
  
  // Combine
  result = combine(leftResult, rightResult);
  
  return result;
}

// Akra-Bazzi Method

// Input: Array A of size n

// Output: Result of the algorithm

function akraBazzi(A, n) {
  // Base case
  if (n == 1) {
    return A[0];
  }
  
  // Divide
  mid = n/2;
  left = A[0..mid-</b>

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 4814 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 12 tokens


Response: <b>

F LOYD -WARSHALL .W; n/
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20

let D .0/ D dij.0/ be a new n  n matrix
for i D 1 to n
for j D 1 to n
if i == j or wij D 1
dij.0/ D 0
else dij.0/ D 
for k D 1 to n
let D .k/ D dij.k/ be a new n  n matrix
for i D 1 to n
for j D 1 to n
dij.k/ D min fdij.k1/ ; dik.k1/ C dkj.k1/ g
if dij.k/ < dij.k1/
�ij.k/ D k
for i D 1 to n
for j D 1 to n
if dij.n/ < 
return D .n/
else return</b>

INFO:openai:error_code=None error_message="'$.input' is invalid. Please check the API reference: https://platform.openai.com/docs/api-reference." error_param=None error_type=invalid_request_error message='OpenAI API error received' stream_error=False
INFO:openai:error_code=None error_message="'$.input' is invalid. Please check the API reference: https://platform.openai.com/docs/api-reference." error_param=None error_type=invalid_request_error message='OpenAI API error received' stream_error=False
INFO:openai:error_code=None error_message="'$.input' is invalid. Please check the API reference: https://platform.openai.com/docs/api-reference." error_param=None error_type=invalid_request_error message='OpenAI API error received' stream_error=False
INFO:openai:error_code=None error_message="'$.input' is invalid. Please check the API reference: https://platform.openai.com/docs/api-reference." error_param=None error_type=invalid_request_error message='OpenAI API error received' stream_error=Fa

KeyboardInterrupt: 